# Lab 3 Group:
Anthony Marcozzi
Courtney Duzet
Sean McNulty
Ed Shokur

## In class exercise: Gene Sequence Clustering

### Training a Markov model
Load the file genes\_training.p, which is available in this homework archive.  genes\_training.p contains 2000 sequences, with each sequence $\mathbf{s}$ consisting of 20 nucleobases $s_i \in \mathrm{Nu},\; \mathrm{Nu} = \{A,T,G,C\}$.  Each of these sequences is generated from one of two separate Markov processes.  The label (aka class) of the process that generated the sequence is given in the dataset.

Learn the Markov model for each class given the training data.  **To do this, for each class compute the prior probability $\mathbf{\pi}_c$ of each nucleobase (the relative frequency of each nucleobase for each class, a vector of length 4) and the matrix of transition probabilities**
$$
\mathcal{A}_{c,kj} = P(s_{i+1}=\mathrm{Nu}_j|s_{i}=\mathrm{Nu}_k),
$$
which is a 4 by 4 matrix.  As a quick sanity check, each row of $\mathcal{A}_c$ should sum to one.  **Using these priors and transition matrices, write a function that generates a new sequence given the class**, i.e. simulates a data point.

In [39]:
import pickle
import numpy as np

# Define some useful constants
N_nucleobases = 4
N_classes = 2
nucleobases = ['A','T','G','C']

# Load the training data using pickle
sequences,labels = pickle.load(open('04-Markov-Models-master/genes_training.p','rb'))
labels = np.array(labels)

# Initialize the class priors and transition matrices
A_0 = np.zeros((N_nucleobases,N_nucleobases))
A_1 = np.zeros((N_nucleobases,N_nucleobases))

##### Train prior #####

#! Compute class priors
pi_0 = labels[labels==0].size / labels.size
pi_1 = 1 - pi_0

#! Compute unconditional nucleobase probabilities
nucleobase_priors = {'A': 0, 'T': 0, 'C': 0, 'G': 0}
num_chars = 0
for seq in sequences:
    for char in seq:
        nucleobase_priors[char] += 1
        num_chars += 1

# Convert the nucleobase count to a probability
nucleobase_priors['A'] /= num_chars
nucleobase_priors['T'] /= num_chars
nucleobase_priors['C'] /= num_chars
nucleobase_priors['G'] /= num_chars

# Compute conditional nucelobase probabilities
pi_0 = {'A': 0, 'T': 0, 'C': 0, 'G': 0}
pi_1 = {'A': 0, 'T': 0, 'C': 0, 'G': 0}
num_0 = 0
num_1 = 0
for i in range(len(sequences)):
    seq = sequences[i]
    label = labels[i]
    for char in seq:
        if label == 0:
            pi_0[char] += 1
            num_0 += 1
        else:
            pi_1[char] += 1
            num_1 += 1
            
# Convert the nucleobase count to a probability
pi_0['A'] /= num_0
pi_0['T'] /= num_0
pi_0['C'] /= num_0
pi_0['G'] /= num_0
pi_1['A'] /= num_1
pi_1['T'] /= num_1
pi_1['C'] /= num_1
pi_1['G'] /= num_1

##### Train transition matrix #####
nbd = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
for s,l in zip(sequences,labels):
    sequence_length = len(s)
    for p in range(sequence_length-1):
        #! s is a length 20 sequence of nucleoboases, for all s, count the number of times that a nucleobase
        #! transitions to another nucleobase and record this information in the appropriate transition matrix (A_0 or A_1)
        n1 = nbd[s[p]]
        n2 = nbd[s[p+1]]
        if l == 0:
            A_0[n1, n2] += 1
        else:
            A_1[n1, n2] += 1

# Convert from counts to probabilities by row normalization
A_0/=A_0.sum(axis=1)[:,np.newaxis]
A_1/=A_1.sum(axis=1)[:,np.newaxis]

##### Generate a synthetic sequence #####
def generate_new_sequence(A,pi,n=20):
    """
    Arguments:
    A -> Nucleobase transition matrix
    pi -> Prior
    n -> length of sequence to generate
    """
    # Draw from the prior for the first nucleobase
    s = [np.random.choice(nucleobases,p=pi)]
    #! Write the code that uses the transition matrix to produce a length n sample
    for i in range(1, n):
        index = nbd[s[i-1]]
        new = np.random.choice(nucleobases, p=A[index, :])
        s.append(new)
    return s

priors = [nucleobase_priors['A'], nucleobase_priors['T'], nucleobase_priors['C'], nucleobase_priors['G']]
generate_new_sequence(A_1, priors, 20)

['G',
 'A',
 'G',
 'A',
 'C',
 'A',
 'A',
 'C',
 'A',
 'C',
 'A',
 'G',
 'T',
 'A',
 'C',
 'A',
 'G',
 'A',
 'C',
 'C']

### A Markov classifier
Having the prior and transition probabilities gives you the ability to evaluate the likelihood of a sequence for a given class as:
$$
P(\mathbf{s}) = P(s_1|\mathbf{\pi}_c) \prod_{i=1}^{n-1} P(s_{i+1}|s_{i},\mathcal{A}_c),
$$
where $\mathbf{\pi}_c$ is the class-conditioned prior probability, and $\mathcal{A}_c$ is the class-conditioned transition matrix.  Comparing the computed likelihood for a given sequence between different classes forms the basis of a classifier in a very similar manner to naive Bayes.  The difference this time is that now each random variable depends on the one before it in the sequence, whereas in naive Bayes we assumed that all the random variables were independent.

Load the file genes\_test.p, which is similar to genes\_training.p.  **For each sequence, compute the likelihood for both classes and assign a label.  Compare this predicted label to the known one, and report the test set accuracy**.  As a point of comparison, my implementation achieved 98.7\% accuracy.

In [55]:
sequences_test,labels_test = pickle.load(open('genes_test.p','rb'))
sequence_probabilities_0 = []
sequence_probabilities_1 = []

for s in sequences_test:
    #! Write a function that evaluates the probability of class membership for each class by multiplying the
    #! prior by the likelihood over all symbol transitions
    p0 = pi_0[s[0]]
    p1 = pi_1[s[0]]
    for i in range(1, len(s)):
        prev = nbd[s[i-1]]
        new = nbd[s[i]]
        p0 *= A_0[prev, new]
        p1 *= A_1[prev, new]
    norm = p0 + p1
    p0 /= norm
    p1 /= norm
    sequence_probabilities_0.append(p0)
    sequence_probabilities_1.append(p1)

predictions = np.column_stack([sequence_probabilities_0, sequence_probabilities_1])

# Now test our results
successes = 0
for i in range(len(labels_test)):
    test_label = labels_test[i]
    pred = predictions[i, :].argmax()
    if pred == test_label:
        successes += 1
percent_correct = successes / len(labels_test) * 100

print(f'Our model predicts {percent_correct}% correctly?')

[[3.72745986e-02 9.62725401e-01]
 [1.58685904e-03 9.98413141e-01]
 [2.32172519e-07 9.99999768e-01]
 ...
 [9.98393703e-01 1.60629713e-03]
 [5.28046553e-06 9.99994720e-01]
 [9.85088650e-01 1.49113504e-02]]
Our model predicts 98.6% correctly?


**Turn in a document with the names of those with whom you worked, an example sequence generated by your model for each class, and a statement of your classifier's overall accuracy.**